In [1]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import pandas as pd
from langfuse.callback import CallbackHandler
import os

load_dotenv()

handler = CallbackHandler(os.environ.get("LANGFUSE_PUBLIC_KEY"), os.environ.get("LANGFUSE_SECRET_KEY"))

In [2]:
MODEL = "gpt-3.5-turbo"

OUTPUT_FOLDER = "../generated_GUIs/"

IMAGE_INPUT_FOLDER = "../data/"

CREATION_PROMPT = PromptTemplate(
    input_variables=["summary"],
    template= """create the HTML code for a Mobilepage, 
    with a bit of styling, that represents following functionality: 
    {summary}."""
)

SUMMARY_PROMPT = PromptTemplate(
    input_variables=["summaries"],
    template="""Given a collection of Screen summaries, which describe the same screen:
        {summaries}.
        
        Extract the provided information of the screen and formulate a new description. make sure, that no information is lost"""
)

S2W = pd.read_csv("../data/Screen2Words.csv")
S2W = S2W.drop('Unnamed: 0', axis=1)

In [3]:
#function for Creating a GUI based on a given description
def create_gui(id, summary, prompt=CREATION_PROMPT, temp=0):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)

    chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    print(response)
    with open(OUTPUT_FOLDER + f"{id}.html", 'w') as f:
        f.write(response)


In [4]:
DESCRIPTION = "The displayed mobile screen is the home page of an application themed around the concept of 'Pitch', presenting a grid of interactive buttons with numbers for selection. This interface contains a variety of numbers including multiple instances of '0', '1', and '2' set against a vibrant pink background, offering users the functionality to perhaps select or input a pitch value. At the bottom of the screen, there is also an advertisement for 'Gutter and Downspout Cleaning' and 'full service lawn and landscape care' from Pride-Outdoor.com."

create_gui(377, DESCRIPTION)

<!DOCTYPE html>
<html>
<head>
    <title>Pitch Application</title>
    <style>
        body {
            background-color: #FF69B4;
        }
        
        .container {
            display: grid;
            grid-template-columns: repeat(3, 1fr);
            gap: 10px;
            padding: 20px;
        }
        
        .button {
            background-color: white;
            border: none;
            border-radius: 5px;
            padding: 20px;
            font-size: 24px;
            text-align: center;
            cursor: pointer;
        }
        
        .advertisement {
            background-color: white;
            padding: 10px;
            text-align: center;
            font-size: 18px;
        }
    </style>
</head>
<body>
    <div class="container">
        <button class="button">0</button>
        <button class="button">1</button>
        <button class="button">2</button>
        <button class="button">0</button>
        <button class="button">1</button>
     